# RAG with IlamaIndex + Milvus + Llama @ Replicate

References
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

## Configuration

In [1]:
class MyConfig:
    pass

MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.INPUT_DATA_DIR = 'data/walmart-reports-1/input'

MY_CONFIG.DB_URI = './rag_4_walmart_llamaindex.db'
MY_CONFIG.COLLECTION_NAME = 'llamaindex_walmart_docs'

MY_CONFIG.LLM_MODEL = "meta/meta-llama-3-8b-instruct"

In [2]:
from llama_index.core import SimpleDirectoryReader
import pprint

# load documents
documents = SimpleDirectoryReader(
    # input_dir = './data/10k/input/'
    input_dir = MY_CONFIG.INPUT_DATA_DIR
).load_data()

print (f"Loaded {len(documents)} chunks")

print("Document [0].doc_id:", documents[0].doc_id)
# pprint.pprint (documents[0], indent=4)

Loaded 300 chunks
Document [0].doc_id: b8d7dbb3-c370-4205-b549-06cc46895ea0


In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/apps/anaconda3/envs/data-prep-kit-dev-1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pymilvus import MilvusClient

client = MilvusClient(MY_CONFIG.DB_URI)

# if we already have a collection, clear it first
if client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)

In [6]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore


vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

✅ Connected to Milvus instance:  ./rag_4_walmart_llamaindex.db


In [7]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 4.72 s, sys: 279 ms, total: 5 s
Wall time: 4.28 s


In [8]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 4.32 s, sys: 168 ms, total: 4.49 s
Wall time: 3.72 s


In [9]:
# See data in vector db

from pymilvus import MilvusClient
import pprint 

client = MilvusClient(MY_CONFIG.DB_URI)
res = client.list_collections()

print(res)
print ('---------')

res = client.describe_collection(
    collection_name = MY_CONFIG.COLLECTION_NAME
)

pprint.pprint(res)


['llamaindex_walmart_docs']
---------
{'aliases': [],
 'auto_id': False,
 'collection_id': 0,
 'collection_name': 'llamaindex_walmart_docs',
 'consistency_level': 0,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {'max_length': 65535},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             'field_id': 101,
             'name': 'embedding',
             'params': {'dim': 384},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 0,
 'num_shards': 0,
 'properties': {}}


In [10]:
import os
## Load Settings from .env file
from dotenv import find_dotenv, dotenv_values

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

os.environ["REPLICATE_API_TOKEN"] = config.get('REPLICATE_API_TOKEN')

In [11]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

In [12]:
query_engine = index.as_query_engine()
res = query_engine.query("What was Walmart's revenue in 2023?")
print(res)



According to the provided context information, Walmart's revenue in 2023 was $100,983 million.


In [13]:
query_engine = index.as_query_engine()
res = query_engine.query("How many distribution facilities does Walmart have?")
print(res)



According to the provided context information, Walmart has a total of 163 distribution facilities located strategically throughout the U.S.


In [14]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)



I'm happy to help! However, I don't see any information about the moon landing in the provided context. The context appears to be a Form 10-K filing with the Securities and Exchange Commission (SEC) for Walmart Inc. for the fiscal year ended January 31, 2024. There is no mention of the moon landing in this document. If you're looking for information about the moon landing, I'd be happy to try to help you find it elsewhere!
